# Analysis of Colombian Education Data: Cleaning and Feature Engineering

**Objective:** Clean the raw dataset of Colombian educational statistics, handle missing values, and engineer predictive features for a Random Forest Regressor targeting the `DESERCIÓN` (Dropout Rate).

**Author:** Senior Data Scientist

---

### Step 0: Import Libraries and Load Data

First, we import the necessary libraries (`pandas` and `numpy`) and load the raw dataset from `Colombia_education_departamento.csv` into a pandas DataFrame.

In [1]:
import pandas as pd
import numpy as np

# Load the dataset
try:
    df = pd.read_csv('Colombia_education_departamento.csv')
    print("Dataset loaded successfully.")
    print(f"Initial shape: {df.shape}")
    print("\nFirst 5 rows:")
    display(df.head())
except FileNotFoundError:
    print("Error: 'Colombia_education_departamento.csv' not found.")

Dataset loaded successfully.
Initial shape: (462, 37)

First 5 rows:


,AÑO,CÓDIGO_DEPARTAMENTO,DEPARTAMENTO,POBLACIÓN_5_16,TASA_MATRICULACIÓN_5_16,COBERTURA_NETA,COBERTURA_NETA_TRANSICIÓN,COBERTURA_NETA_PRIMARIA,COBERTURA_NETA_SECUNDARIA,COBERTURA_NETA_MEDIA,...,REPROBACIÓN,REPROBACIÓN_TRANSICIÓN,REPROBACIÓN_PRIMARIA,REPROBACIÓN_SECUNDARIA,REPROBACIÓN_MEDIA,REPITENCIA,REPITENCIA_TRANSICIÓN,REPITENCIA_PRIMARIA,REPITENCIA_SECUNDARIA,REPITENCIA_MEDIA
0,2011,5,Antioquia,1288473,94.01%,93.85%,70.28%,94.12%,75.68%,44.37%,...,2.06%,0.07%,94.56%,2.54%,2.96%,4.25%,0.07%,4.56%,5.27%,1.68%
1,2011,8,Atlántico,523935,99.32%,99.05%,50.59%,98.93%,80.22%,50.17%,...,0.54%,0.12%,96.49%,0.67%,0.75%,1.82%,0.12%,1.77%,2.18%,0.88%
2,2011,11,"Bogotá, D.C.",1479334,90.7%,90.29%,68.63%,86.99%,84.7%,55.01%,...,0%,0%,94.69%,0%,0%,3.23%,0%,2.3%,5.11%,2.57%
3,2011,13,Bolívar,496676,91.57%,91.4%,59.74%,90.81%,67.34%,39.17%,...,2.1%,0.46%,95.48%,2.75%,3.67%,4.43%,0.46%,4.44%,5.37%,2.28%
4,2011,15,Boyacá,300501,86.16%,86.11%,63.36%,82.5%,74.65%,49.09%,...,2.73%,0.17%,96.1%,4.31%,3.26%,2.62%,0.17%,1.9%,4.19%,1.55%


### Step 1: Data Cleaning & Typing

In this step, we perform two critical cleaning operations:
1.  **String Cleanup:** We identify all columns that contain string representations of percentages (e.g., "93.85%"). We then strip the '%' symbol, handle decimal commas, and convert these columns to a proper floating-point numeric type for calculations.
2.  **Corrupt Data Removal:** The `SEDES_CONECTADAS_A_INTERNET` column is known to have severe data quality issues and will be dropped entirely as per the project requirements.

In [2]:
print("--- Step 1: Cleaning data and fixing types ---")

# Identify columns with percentage strings
percent_cols = [col for col in df.columns if df[col].dtype == 'object' and df[col].str.contains('%', na=False).any()]

print(f"Found {len(percent_cols)} columns with '%' to convert: {percent_cols[:5]}...")  # Show first 5

# Clean and convert these columns to numeric
for col in percent_cols:
    df[col] = df[col].str.replace('%', '', regex=False).str.replace(',', '.', regex=False)
    df[col] = pd.to_numeric(df[col], errors='coerce')

print("\nConverted percentage columns to numeric type.")

# Drop the corrupt 'SEDES_CONECTADAS_A_INTERNET' column
if 'SEDES_CONECTADAS_A_INTERNET' in df.columns:
    df = df.drop(columns=['SEDES_CONECTADAS_A_INTERNET'])
    print("Dropped 'SEDES_CONECTADAS_A_INTERNET' column.")

print("\nData cleaning and typing complete.")
display(df.head())

--- Step 1: Cleaning data and fixing types ---
Found 31 columns with '%' to convert: ['TASA_MATRICULACIÓN_5_16', 'COBERTURA_NETA', 'COBERTURA_NETA_TRANSICIÓN', 'COBERTURA_NETA_PRIMARIA', 'COBERTURA_NETA_SECUNDARIA']...

Converted percentage columns to numeric type.
Dropped 'SEDES_CONECTADAS_A_INTERNET' column.

Data cleaning and typing complete.


,AÑO,CÓDIGO_DEPARTAMENTO,DEPARTAMENTO,POBLACIÓN_5_16,TASA_MATRICULACIÓN_5_16,COBERTURA_NETA,COBERTURA_NETA_TRANSICIÓN,COBERTURA_NETA_PRIMARIA,COBERTURA_NETA_SECUNDARIA,COBERTURA_NETA_MEDIA,...,REPROBACIÓN,REPROBACIÓN_TRANSICIÓN,REPROBACIÓN_PRIMARIA,REPROBACIÓN_SECUNDARIA,REPROBACIÓN_MEDIA,REPITENCIA,REPITENCIA_TRANSICIÓN,REPITENCIA_PRIMARIA,REPITENCIA_SECUNDARIA,REPITENCIA_MEDIA
0,2011,5,Antioquia,1288473,94.01,93.85,70.28,94.12,75.68,44.37,...,2.06,0.07,94.56,2.54,2.96,4.25,0.07,4.56,5.27,1.68
1,2011,8,Atlántico,523935,99.32,99.05,50.59,98.93,80.22,50.17,...,0.54,0.12,96.49,0.67,0.75,1.82,0.12,1.77,2.18,0.88
2,2011,11,"Bogotá, D.C.",1479334,90.70,90.29,68.63,86.99,84.70,55.01,...,0.00,0.00,94.69,0.00,0.00,3.23,0.00,2.30,5.11,2.57
3,2011,13,Bolívar,496676,91.57,91.40,59.74,90.81,67.34,39.17,...,2.10,0.46,95.48,2.75,3.67,4.43,0.46,4.44,5.37,2.28
4,2011,15,Boyacá,300501,86.16,86.11,63.36,82.50,74.65,49.09,...,2.73,0.17,96.10,4.31,3.26,2.62,0.17,1.90,4.19,1.55


### Step 2: Smart Imputation (Handling Nulls)

Missing data can break our model. We'll handle it intelligently:
1.  **Target Variable (`DESERCIÓN`):** Any row where the target variable is missing is unusable for training. We will drop these rows.
2.  **`TAMAÑO_PROMEDIO_DE_GRUPO` (Class Density):** This is a potentially high-value feature. Instead of dropping rows, we will impute missing values using the **median** value for that specific department (`CÓDIGO_DEPARTAMENTO`). If a department has no data at all for this column, we'll use the global median as a fallback.

In [7]:
initial_rows = len(df)
df.dropna(subset=['DESERCIÓN'], inplace=True)
print(f"Dropped {initial_rows - len(df)} rows with missing target (DESERCIÓN).")

# ---------------------------------------------------------
# 3. SMART IMPUTATION & OUTLIER FIX (Class Density)
# ---------------------------------------------------------
# Rename for easier typing
if 'TAMAÑO_PROMEDIO_DE_GRUPO' in df.columns:
    df.rename(columns={'TAMAÑO_PROMEDIO_DE_GRUPO': 'classroom_density'}, inplace=True)

# A. Impute missing values using the Median of THAT Department
# (Better than global average because rural vs urban classes differ size)
df['classroom_density'] = df.groupby('CÓDIGO_DEPARTAMENTO')['classroom_density'].transform(
    lambda x: x.fillna(x.median())
)
# Fallback: If a department has NO data at all, use global median
global_median = df['classroom_density'].median()
df['classroom_density'].fillna(global_median, inplace=True)

# B. THE CRITICAL FIX: The "Comma vs Dot" Error
# Problem: Values like 30588.00 should be 30.588
# Logic: Any class size > 100 is physically impossible. Divide by 1000.
outlier_mask = df['classroom_density'] > 100
num_outliers = outlier_mask.sum()
if num_outliers > 0:
    print(f"Found {num_outliers} rows with impossible class sizes (e.g., >100). Fixing scaling...")
    df.loc[outlier_mask, 'classroom_density'] = df.loc[outlier_mask, 'classroom_density'] / 1000

Dropped 0 rows with missing target (DESERCIÓN).
Found 56 rows with impossible class sizes (e.g., >100). Fixing scaling...


/var/folders/qv/w6gmqgrs64b5x2c7vhrcz92w0000gr/T/ipykernel_28953/3626047835.py:19: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['classroom_density'].fillna(global_median, inplace=True)


### Step 3: Feature Engineering ("The Golden Features")

Now, we create powerful new predictors based on domain knowledge:
1.  **`over_age_gap`**: Calculated as `COBERTURA_BRUTA - COBERTURA_NETA`. This feature captures the percentage of students who are over-age for their grade level, a strong indicator of social risk and potential dropout.
2.  **`repitencia_lag_1`**: Dropout is often preceded by academic failure. This feature captures the repetition rate from the *previous year* for the same department. We sort the data by department and year to create this lagged feature.
3.  **`classroom_density`**: We rename the cleaned `TAMAÑO_PROMEDIO_DE_GRUPO` column to a more intuitive name.

In [8]:
print("--- Step 3: Engineering new features ---")

# Feature 1: over_age_gap
if 'COBERTURA_BRUTA' in df.columns and 'COBERTURA_NETA' in df.columns:
    df['over_age_gap'] = df['COBERTURA_BRUTA'] - df['COBERTURA_NETA']
    print("Created 'over_age_gap' feature.")
    print(f"  Mean: {df['over_age_gap'].mean():.2f}, Std: {df['over_age_gap'].std():.2f}")

# Feature 2: repitencia_lag_1
if 'REPITENCIA' in df.columns:
    # Sort data to ensure correct lagging within groups
    df = df.sort_values(by=['CÓDIGO_DEPARTAMENTO', 'AÑO'])
    df['repitencia_lag_1'] = df.groupby('CÓDIGO_DEPARTAMENTO')['REPITENCIA'].shift(1)
    print("Created 'repitencia_lag_1' (lagged repetition) feature.")
    print(f"  NaN values created: {df['repitencia_lag_1'].isnull().sum()} (first year of each department)")

# Feature 3: classroom_density
if 'TAMAÑO_PROMEDIO_DE_GRUPO' in df.columns:
    df = df.rename(columns={'TAMAÑO_PROMEDIO_DE_GRUPO': 'classroom_density'})
    print("Renamed 'TAMAÑO_PROMEDIO_DE_GRUPO' to 'classroom_density'.")

print("\nFeature engineering complete.")
print("\nSample of new features:")
display(df[['AÑO', 'DEPARTAMENTO', 'over_age_gap', 'REPITENCIA', 'repitencia_lag_1', 'classroom_density', 'DESERCIÓN']].head(20))

--- Step 3: Engineering new features ---
Created 'over_age_gap' feature.
  Mean: 12.26, Std: 3.43
Created 'repitencia_lag_1' (lagged repetition) feature.
  NaN values created: 33 (first year of each department)

Feature engineering complete.

Sample of new features:


,AÑO,DEPARTAMENTO,over_age_gap,REPITENCIA,repitencia_lag_1,classroom_density,DESERCIÓN
0,2011,Antioquia,12.96,4.25,NaN,27.470,3.97
33,2012,Antioquia,13.15,1.93,4.25,27.960,4.01
66,2013,Antioquia,13.76,1.57,1.93,52.720,3.57
99,2014,Antioquia,13.05,2.09,1.57,31.170,2.33
132,2015,Antioquia,13.35,2.83,2.09,30.390,3.82
165,2016,Antioquia,13.64,3.02,2.83,30.588,4.47
198,2017,Antioquia,12.15,6.52,3.02,31.728,4.15
231,2018,Antioquia,14.28,3.13,6.52,31.170,3.97
264,2019,Antioquia,13.74,3.19,3.13,31.170,4.02
297,2020,Antioquia,12.76,6.48,3.19,31.170,2.30


### Step 4: Leakage Prevention & Final Cleaning

To prevent our model from "cheating," we must remove any columns that would give away the answer (`DESERCIÓN`). These include:
- `APROBACIÓN` (Approval Rate) and its sub-columns.
- `REPROBACIÓN` (Failure Rate) and its sub-columns.

Additionally, creating a lagged feature introduces `NaN` values for the first year of each department's data. These rows are now unusable and must be dropped.

In [9]:
print("--- Step 4: Preventing data leakage ---")

# Find and drop all columns related to approval and failure rates
leakage_cols = [col for col in df.columns if 'APROBACIÓN' in col or 'REPROBACIÓN' in col]
print(f"Identified {len(leakage_cols)} leakage columns:")
print(leakage_cols[:10])  # Show first 10
df = df.drop(columns=leakage_cols)
print(f"\nDropped {len(leakage_cols)} leakage columns.")

# Drop rows with NaN values created by the lagging process
initial_rows = len(df)
df.dropna(subset=['repitencia_lag_1'], inplace=True)
print(f"Dropped {initial_rows - len(df)} rows with NaN values from lagged features.")

print("\nLeakage prevention and final cleaning complete.")
print(f"Final dataset shape before splitting: {df.shape}")

--- Step 4: Preventing data leakage ---
Identified 10 leakage columns:
['APROBACIÓN', 'APROBACIÓN_TRANSICIÓN', 'APROBACIÓN_PRIMARIA', 'APROBACIÓN_SECUNDARIA', 'APROBACIÓN_MEDIA', 'REPROBACIÓN', 'REPROBACIÓN_TRANSICIÓN', 'REPROBACIÓN_PRIMARIA', 'REPROBACIÓN_SECUNDARIA', 'REPROBACIÓN_MEDIA']

Dropped 10 leakage columns.
Dropped 33 rows with NaN values from lagged features.

Leakage prevention and final cleaning complete.
Final dataset shape before splitting: (429, 28)


### Step 5: Final Preparation & Verification

Finally, we'll:
1.  **Verify Signal:** Check the correlation between our newly engineered features and the target variable `DESERCIÓN`. This helps confirm that our new features are indeed related to the dropout rate.
2.  **Prepare `X` and `y`:** Split the data into our final features matrix (`X`) and the target vector (`y`), ready for training a machine learning model.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

print("--- Step 6: Training the Optimized Model ---")

# 1. SELECT ONLY THE BEST FEATURES
# We discard the raw 'Cobertura' columns because 'over_age_gap' captures their essence better.
# We keep Department ID because geography matters.
predictors = [
    'over_age_gap', 
    'repitencia_lag_1', 
    'classroom_density', 
    'POBLACIÓN_5_16', 
    'CÓDIGO_DEPARTAMENTO'  # Ensure this column is in your df_final!
]

# Verify these columns exist
available_predictors = [col for col in predictors if col in df_final.columns]
print(f"Training with features: {available_predictors}")

X = df_final[available_predictors]
y = df_final['DESERCIÓN']

# 2. SPLIT DATA
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 3. TRAIN RANDOM FOREST
# n_estimators=200: More trees = smoother predictions
rf_model = RandomForestRegressor(n_estimators=200, max_depth=10, random_state=42)
rf_model.fit(X_train, y_train)

# 4. EVALUATE
y_pred = rf_model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
print(f"\nModel MAE: {mae:.4f} (Average prediction error in % points)")

# 5. VISUALIZE: ACTUAL vs PREDICTED (The "Blob" Check)
plt.figure(figsize=(10, 6))
sns.scatterplot(x=y_test, y=y_pred, alpha=0.6)
plt.plot([0, y_test.max()], [0, y_test.max()], 'r--', lw=2) # Perfect prediction line
plt.xlabel('Actual Dropout Rate (%)')
plt.ylabel('Predicted Dropout Rate (%)')
plt.title('Actual vs Predicted: Did we fix the blob?')
plt.show()

# 6. VISUALIZE: FEATURE IMPORTANCE (The "Why")
importances = rf_model.feature_importances_
feature_df = pd.DataFrame({'Feature': available_predictors, 'Importance': importances})
feature_df = feature_df.sort_values(by='Importance', ascending=False)

plt.figure(figsize=(10, 4))
sns.barplot(x='Importance', y='Feature', data=feature_df, hue='Feature', palette='viridis')
plt.title('What actually drives Dropout?')
plt.show()

--- Step 6: Training the Optimized Model ---


NameError: name 'df_final' is not defined

### Summary

✅ **Data Cleaning:** Converted percentage strings to numeric, dropped corrupt column  
✅ **Imputation:** Handled missing values intelligently using departmental medians  
✅ **Feature Engineering:** Created 3 "Golden Features" based on domain expertise  
✅ **Leakage Prevention:** Removed all columns that could leak the target  
✅ **Ready for Training:** Final X and y dataframes prepared for Random Forest Regressor

---

**Next Steps:**
- Train a Random Forest Regressor using `X` and `y`
- Evaluate model performance with cross-validation
- Analyze feature importance to understand dropout drivers